# Solution 1

# Importing necessary packages

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import math
import numpy as np
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Parameters

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 100

# loading in and preprocessing the train and test datasets

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "archive/train",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 3115 files belonging to 36 classes.


In [4]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "archive/test",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 359 files belonging to 36 classes.


In [5]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "archive/validation",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 351 files belonging to 36 classes.


# Solution 2

# Build a simple convolutional neural network(CNN) model

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 10
num_classes = 36

# Set the path to your dataset
train_data_dir = 'archive/train'
validation_data_dir = 'archive/validation'
test_data_dir = 'archive/test'

# Data augmentation for better generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


# Load test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.
Found 359 images belonging to 36 classes.


In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compiling using Adam Optimizer

In [12]:

model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model

history = history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

In [18]:
validation_steps = validation_generator.samples // BATCH_SIZE
validation_predictions = model.predict(validation_generator, steps=validation_steps)

# Get true labels
true_labels = validation_generator.classes

# Convert true labels to one-hot encoding
true_labels_one_hot = tf.keras.utils.to_categorical(true_labels, num_classes=num_classes)

# Create a confusion matrix
conf_matrix = confusion_matrix(np.argmax(true_labels_one_hot, axis=1), 
                               np.argmax(validation_predictions, axis=1),
                                labels = np.arange(num_classes))


10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 734ms/step


ValueError: Found input variables with inconsistent numbers of samples: [351, 320]

In [ ]:
conf_matrix

In [ ]:
# Evaluate the model on the validation set
validation_steps = validation_generator.samples // BATCH_SIZE
validation_loss, validation_accuracy = model.evaluate(validation_generator, steps=validation_steps)

# Generate predictions on the validation set
validation_predictions = model.predict(validation_generator, steps=validation_steps)

# Get true labels
true_labels = validation_generator.classes

# Convert predictions to class labels
predicted_labels = np.argmax(validation_predictions, axis=1)

# Create a confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()